<a href="https://colab.research.google.com/drive/1dUUN8StWSvQiVKF_4GzrmVuh6f1EeBCG#scrollTo=8AoNDCXvlgYe" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Минзадинов Ислам гр 6133

## Задание на Лабораторную работу №1

### Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):
1. Найти велосипед с максимальным временем пробега.

2. Найти наибольшее геодезическое расстояние между станциями.

3. Найти путь велосипеда с максимальным временем пробега через станции.

4. Найти количество велосипедов в системе.

5. Найти пользователей потративших на поездки более 3 часов.

### 1. Настройка окружения

In [2]:
# установка pyspark
!pip install pyspark

In [3]:
# Добавляю гугл диск с данными
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pyspark
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime
from functools import reduce

### 2. Создаю проект и проверяю версию, чтобы убедиться в работоспособности

In [5]:
# Создание точки конфигурации
sc = SparkContext(conf=SparkConf().setAppName('L1ItroductionToApacheSpark').setMaster("local[*]"))
# Проверка установки
sc

<SparkContext master=local[*] appName=L1ItroductionToApacheSpark>

### 3. Загрузка данных

In [6]:
data_trip = sc.textFile("/content/drive/MyDrive/BigData/L1/trips.csv")
data_station = sc.textFile("/content/drive/MyDrive/BigData/L1/stations.csv")

### 4. Предобработка данных

In [7]:
headerOfTrips = data_trip.first()
headerOfStations = data_station.first()

In [8]:
tripsWithOutHeader = data_trip.filter(lambda row: row != headerOfTrips)
stationsWithOutHeader = data_station.filter(lambda row: row != headerOfStations)

In [9]:
tripsWithOutHeader.take(10)

['4576,63,,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127',
 '4607,,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138',
 '4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214',
 '4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060',
 '4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103',
 '4927,103,8/29/2013 18:54,Golden Gate at Polk,59,8/29/2013 18:56,Golden Gate at Polk,59,527,Subscriber,94109',
 '4500,109,8/29/2013 13:25,Santa Clara at Almaden,4,8/29/2013 13:27,Adobe on Almaden,5,679,Subscriber,95112',
 '4563,111,8/29/2013 14:02,San Salvador at 1st,8,8/29/2013 14:04,San Salvador at 1st,8,687,Subscriber,95112',
 '4760,113,8/29/2013 17:01,South Van Ness at Market,66,8/29/2013 17:03,South Van Ness at Market,66,553,Subscriber,9410

In [10]:
stationsWithOutHeader.take(10)

['2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013',
 '3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013',
 '4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013',
 '5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013',
 '6,San Pedro Square,37.336721000000004,-121.894074,15,San Jose,8/7/2013',
 '7,Paseo de San Antonio,37.333798,-121.88694299999999,15,San Jose,8/7/2013',
 '8,San Salvador at 1st,37.330165,-121.88583100000001,15,San Jose,8/5/2013',
 '9,Japantown,37.348742,-121.89471499999999,15,San Jose,8/5/2013',
 '10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013',
 '11,MLK Library,37.335885,-121.88566000000002,19,San Jose,8/6/2013']

In [11]:
trip_parametr = list(enumerate(headerOfTrips.split(",")))
station_parametr = list(enumerate(headerOfStations.split(",")))

In [12]:
final_trips_data = tripsWithOutHeader.map(lambda x: x.split(','))
final_stations_data = stationsWithOutHeader.map(lambda x: x.split(','))

In [13]:
final_trips_data.take(1)

[['4576',
  '63',
  '',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127']]

In [14]:
final_stations_data.take(1)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013']]

### 3. Разработка и применения модели.

Модель для trips.

In [15]:
trip_parametr

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [16]:
def initMT(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str

    for trip in trips:
        try:
            yield Trip(
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            )
        except:
            pass

In [17]:
station_parametr

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

Модель для Stantions.

In [18]:
def initMS(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str

    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

Применени модели к данным

In [19]:
tripsProcessedByModel = final_trips_data.mapPartitions(initMT)
tripsProcessedByModel.take(3)

[Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214'),
 Trip(trip_id=4251, duration=77, start_date=datetime.datetime(2013, 8, 29, 11, 29), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 11, 30), end_station_name='San Jose City Hall', end_station_id='10', bike_id=26, subscription_type='Subscriber', zip_code='95060'),
 Trip(trip_id=4299, duration=83, start_date=datetime.datetime(2013, 8, 29, 12, 2), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 12, 4), end_station_name='Market at 10th', end_station_id='67', bike_id=319, subscription_type='Subscriber', zip_code='94103')]

In [20]:
stationsProcessedByModel = final_stations_data.mapPartitions(initMS)
stationsProcessedByModel.take(3)

[Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
 Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0)),
 Station(station_id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dockcount=11, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))]

## 4. Задания

### Задание 1. Найти велосипед с максимальным временем пробега.

In [21]:
tripMapWithBicycle = tripsProcessedByModel.keyBy(lambda x: x.bike_id)
durationBicycleMap = tripMapWithBicycle.mapValues(lambda x: x.duration).reduceByKey(lambda x1, x2: x1 + x2)
topOfDurationBicycleMap = durationBicycleMap.top(1, key=lambda x: x[1])[0][0]
topOfDurationBicycleMap

535

### Задание 2. Найти наибольшее геодезическое расстояние между станциями.

In [22]:
from math import radians, sin, cos, sqrt

def calculate_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371.0088

    lat1, lon1 = radians(lat1), radians(lon1)
    lat2, lon2 = radians(lat2), radians(lon2)

    diff_lat = lat2 - lat1
    diff_lon = lon2 - lon1

    hav_lat = sin(diff_lat / 2) ** 2
    hav_lon = sin(diff_lon / 2) ** 2

    h = hav_lat + cos(lat1) * cos(lat2) * hav_lon

    distance = 2 * earth_radius * sqrt(h)
    return distance

# Создаем пары станций с расстояниями между ними
station_pairs = stationsProcessedByModel.cartesian(stationsProcessedByModel) \
    .filter(lambda x: x[0].station_id != x[1].station_id) \
    .map(lambda x: (x[0], x[1], calculate_distance(x[0].lat, x[0].long, x[1].lat, x[1].long)))

# Находим пару станций с максимальным расстоянием
max_distance_pair = station_pairs.max(key=lambda x: x[2])

way_between_stations = (max_distance_pair[0].name, max_distance_pair[1].name)
kilometers_between_stations = max_distance_pair[2]

print(f"Максимальное расстояние между станциями: {way_between_stations}, {kilometers_between_stations} км.")

Максимальное расстояние между станциями: ('SJSU - San Salvador at 9th', 'Embarcadero at Sansome'), 69.92062162445784 км.


### Задание 3. Найти путь велосипеда с максимальным временем пробега через станции.

In [23]:
wayBicycleWithMaxDuration = tripsProcessedByModel.filter(lambda x: x.bike_id == topOfDurationBicycleMap)\
.sortBy(lambda x: x.start_date).map(lambda x: (x.start_station_name, x.end_station_name))
wayBicycleWithMaxDuration.take(1)

[('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)')]

### Задание 4. Найти количество велосипедов в системе.

In [24]:
countOfBicycle = tripsProcessedByModel.map(lambda x: x.bike_id).distinct().count()
countOfBicycle

700

### Задание 5. Найти пользователей потративших на поездки более 3 часов.

In [25]:
threeHours = 3*60*60
usersOfBicycle = tripsProcessedByModel.filter(lambda x: x.duration > threeHours)\
.map(lambda x: x.zip_code).filter(lambda x: x != "").distinct()
print('5 пользователей потративших на поездки более 3 часов: ', usersOfBicycle.take(5))

5 пользователей потративших на поездки более 3 часов:  ['58553', '94301', '94039', '94133', '93726']
